In [95]:
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
import math
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA
from sklearn.decomposition import KernelPCA
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.manifold import LocallyLinearEmbedding

# Singular Value Decomposition (SVD)
X1 = 2 * np.random.rand(100,1)
X2 = 4 + 3 * X1 + np.random.rand(100,1)
X = np.c_[X1, X2]
X = X - X.mean(axis=0)
U, s, Vt = np.linalg.svd(X)
W2 = Vt.T[:,:2]
X2D = X.dot(W2)
plt.figure(figsize=(10,5))
plt.subplot(121)
plt.plot(X[:,0], X[:,1], ".", label="data")
plt.plot(np.arange(-3,3,0.01), np.arange(-3,3,0.01)*Vt.T[:,0][1]/Vt.T[:,0][0], 
         "r--", label="PCA 1")
plt.plot(np.arange(-3,3,0.01), np.arange(-3,3,0.01)*Vt.T[:,1][1]/Vt.T[:,1][0], 
         "g--", label="PCA 2")
plt.legend()
plt.xlabel("X_1")
plt.ylabel("X_2")
plt.xlim([-3,3])
plt.ylim([-3,3])
plt.title("PCA")
plt.subplot(122)
plt.plot(X2D[:,0]*math.cos(math.atan(Vt.T[:,0][1]/Vt.T[:,0][0])), 
         X2D[:,0]*math.sin(math.atan(Vt.T[:,0][1]/Vt.T[:,0][0])), 
         "r.", label="data 1")
plt.plot(X2D[:,1]*math.cos(math.atan(Vt.T[:,1][1]/Vt.T[:,1][0])), 
         X2D[:,1]*math.sin(math.atan(Vt.T[:,1][1]/Vt.T[:,1][0])), 
         "g.", label="data 2")
plt.legend()
plt.xlabel("X_1")
plt.ylabel("X_2")
plt.xlim([-3,3])
plt.ylim([-3,3])
plt.title("Project")
plt.savefig("../plots/ex_8_01.pdf")

# PCA
pca = PCA(n_components = 2)
X2D = pca.fit_transform(X)
pca.components_.T
pca.explained_variance_ratio_

# determine number of dimensions: #1
pca = PCA()
pca.fit(X)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1
pca = PCA(n_components = d)
X2D = pca.fit_transform(X)

# determine number of dimensions: #2
pca = PCA(n_components = 0.95)
X2D = pca.fit_transform(X)

# read MNIST data
df_train = pd.read_csv("../datasets/mnist_train.csv")
df_test = pd.read_csv("../datasets/mnist_test.csv")
X_train = df_train.drop("5", axis=1).values
y_train = df_train["5"].values.ravel()

# reduce dimension
pca = PCA()
pca.fit(X_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1
plt.figure()
plt.plot(range(len(cumsum)), cumsum, "b-")
plt.plot(range(len(cumsum)), [0.95]*len(cumsum), "r--")
plt.plot(range(len(cumsum)), [1.00]*len(cumsum), "k--")
plt.xlabel("number of dimensions")
plt.ylabel("explained variance ratio")
plt.ylim([0.4,1.1])
plt.savefig("../plots/ex_8_02.pdf")

# PCA for compresion
pca = PCA(n_components=d)
X_reduced = pca.fit_transform(X_train)
X_recovered = pca.inverse_transform(X_reduced)
plt.figure(figsize=(10,5))
plt.subplot(121)
plt.imshow(X_train[100,:].reshape(28,28), cmap=plt.get_cmap("Greys"))
plt.title("Raw")
plt.subplot(122)
plt.imshow(X_recovered[100,:].reshape(28,28), cmap=plt.get_cmap("Greys"))
plt.title("Compressed")
plt.savefig("../plots/ex_8_03.pdf")

# incremental PCA
n_batches = 100
inc_pca = IncrementalPCA(n_components=154)
for X_batch in np.array_split(X_train, n_batches):
    inc_pca.partial_fit(X_batch)
X_reduced_inc = inc_pca.transform(X_train)

# randomized PCA
rnd_pca = PCA(n_components=154, svd_solver="randomized")
X_reduced_rnd = rnd_pca.fit_transform(X_train)
"""

# read swiss roll
X, y = make_swiss_roll(n_samples=2000, noise=0.2, random_state=42)
"""
# kernel PCA
lin_pca = KernelPCA(n_components=2, kernel="linear", gamma=0.04)
X_lin = lin_pca.fit_transform(X)
rbf_pca = KernelPCA(n_components=2, kernel="rbf", gamma=0.04)
X_rbf = rbf_pca.fit_transform(X)
sig_pca = KernelPCA(n_components=2, kernel="sigmoid", gamma=0.04)
X_sig = sig_pca.fit_transform(X)
plt.figure(figsize=(5,5))
plt.subplot(221)
ax = plt.figure(figsize=(10,10)).add_subplot(221, projection='3d')
ax.scatter(X[:,0], X[:,1], X[:,2], c=y, cmap=plt.get_cmap("jet"))
ax.view_init(15,80)
plt.subplot(222)
plt.scatter(X_lin[:,0], X_lin[:,1], c=y, cmap=plt.get_cmap("jet"))
plt.title("Linear Kernel")
plt.subplot(223)
plt.scatter(X_rbf[:,0], X_rbf[:,1], c=y, cmap=plt.get_cmap("jet"))
plt.title("RBF Kernel")
plt.subplot(224)
plt.scatter(X_sig[:,0], X_sig[:,1], c=y, cmap=plt.get_cmap("jet"))
plt.title("Sigmoid Kernel")
plt.savefig("../plots/ex_8_04.pdf")

# grid search
clf = Pipeline([
    ("kpca", KernelPCA(n_components=2)),
    ("log_reg", LogisticRegression())
])
param_grid = [{
    "kpca__gamma": np.linspace(0.03, 0.05, 10), "kpca__kernel": ["rbf", "sigmoid"]
}]
grid_search = GridSearchCV(clf, param_grid, cv=3, scoring="accuracy")
grid_search.fit(X, (y / 2.).astype(int))
print(grid_search.cv_results_["mean_test_score"])

# reconstruction error
rbf_pca = KernelPCA(n_components=2, kernel="rbf", gamma=0.0433, fit_inverse_transform=True)
X_reduced = rbf_pca.fit_transform(X)
X_preimage = rbf_pca.inverse_transform(X_reduced)
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(221, projection='3d')
ax.scatter(X[:,0], X[:,1], X[:,2], c=y, cmap=plt.get_cmap("jet"))
ax.view_init(15,40)
ax.set_title("Original")
ax = fig.add_subplot(222)
ax.scatter(X_reduced[:,0], X_reduced[:,1], c=y, cmap=plt.get_cmap("jet"))
ax.set_title("RBF PCA")
ax = fig.add_subplot(223, projection='3d')
ax.scatter(X_preimage[:,0], X_preimage[:,1], X_preimage[:,2], c=y, cmap=plt.get_cmap("jet"))
ax.view_init(15,40)
ax.set_title("Pre-image")
fig.savefig("../plots/ex_8_05.pdf")
print(mean_squared_error(X, X_preimage))

# Locally Linear Embedding (LLE)
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(221, projection='3d')
ax.scatter(X[:,0], X[:,1], X[:,2], c=y, cmap=plt.get_cmap("jet"))
ax.view_init(15,80)
ax.set_title("Original")
ax = fig.add_subplot(222)
lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10)
X_reduced = lle.fit_transform(X)
ax.scatter(X_reduced[:,0], X_reduced[:,1], c=y, cmap=plt.get_cmap("jet"))
ax.set_title("LLE")
ax = fig.add_subplot(223)
lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10, method="ltsa")
X_reduced = lle.fit_transform(X)
ax.scatter(X_reduced[:,0], X_reduced[:,1], c=y, cmap=plt.get_cmap("jet"))
ax.set_title("LLE LTSA")
ax = fig.add_subplot(224)
lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10, method="modified")
X_reduced = lle.fit_transform(X)
ax.scatter(X_reduced[:,0], X_reduced[:,1], c=y, cmap=plt.get_cmap("jet"))
ax.set_title("LLE modified")
fig.savefig("../plots/ex_8_06.pdf")